In [ ]:
#!pip install psycopg2 - Maybe you gonna need this to run the sqlalchemy library. If you aren´t using an sql query, ignore this step
import pandas as pd 
import sqlalchemy   - I my case, i´ve used this library to get the data from an SQL script
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from operator import attrgetter
import matplotlib.colors as mcolors
-------------------------------------------------------------------------------------------------
# JUST IN CASE IF YOU NEED RUN AN SQL QUERY. IF YOU AREN´T USING AN SQL QUERY, IGNORE THIS STEP
DB_USER = ('YOUR_USER')
DB_PORT = ('PORT')
DB_NAME = ('DATASET_NAME')
DB_HOST = ('HOST')
DB_PASS = ('PASSWORD')


def get_engine():
    return sqlalchemy.create_engine("postgresql+psycopg2://{}:{}@{}:{}/{}".format(DB_USER,
                                                                                  DB_PASS,
                                                                                  DB_HOST,
                                                                                  int(DB_PORT),
                                                                                  DB_NAME),
                                    connect_args={"sslmode": "prefer"})
query = """

      YOUR SQL QUERY
  
  """

df_base = pd.read_sql(query, get_engine())
------------------------------------------------------------------------------------------------

# THIS STEP GONNA TRANSFORM THE DATE IN A BETTER FORMAT , MAYBE YOU DONT GONNA NEED
df_base['DATE_COLUMN'] = pd.to_datetime(df_base['DATE_COLUMN'],format="%Y/%m")
df_base['DATE_COLUMN'] = [i.strftime('%Y-%m') for i in df_base['DATE_COLUMN']]


df_base = df_base.pivot_table(index = 'INDEX',                    # HERE WE PUT OR INDEX, THE MAIN REFERENCE   
                                     columns = 'COLUMNS',         # ADD THE COLUMNS
                                     values = 'RETENTION_VALUE')  # THE RETENTION VALUE WE ARE LOOKING FOR
#  CREATING THE RETENTION MATRIX                                    
base_size = df_base.iloc[:,0]
retention_matrix = df_base.divide(base_size, axis = 0)                                    

# PLOTNG OUR MATRIX
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(15, 10), sharey=True, gridspec_kw={'width_ratios': [1, 11]})
    
    # retention matrix
    sns.heatmap(retention_matrix, 
                mask=retention_matrix_cohort.isnull(), 
                annot=True, 
                fmt='.0%', 
                cmap='RdYlGn', 
                ax=ax[1])
    ax[1].set_title('YOUR TITLE HERE', fontsize=16)
    ax[1].set(xlabel='# of periods',
              ylabel='')

    # base size
    base_size_df = pd.DataFrame(base_size).rename(columns={0: 'base_size'})
    white_cmap = mcolors.ListedColormap(['white'])
    sns.heatmap(base_size_df, 
                annot=True, 
                cbar=False, 
                fmt='g', 
                cmap=white_cmap, 
                ax=ax[0])